In [1]:
# Imports and global variables
import requests
import json
import pandas
from dateutil import parser

BASE_URL = "http://localhost:8080/engine-rest"

In [2]:
# CLEANUP
# delete all process definitions and running instances
# ! we need to delete all the running instances first

def status_code_successful(status_code: int):
    return str(status_code)[0] == '2'


def delete_all_data(target: str):
    POSSIBLE_TARGETS = ["process-instance", "process-definition", "deployment", "decision-definition"]
    if target not in POSSIBLE_TARGETS:
        raise Exception(str(target) + "not a valid data deletion target")
        
    get_response = requests.get(BASE_URL + "/" + target)
    assert(status_code_successful(get_response.status_code))
    #print(json.dumps(get_response.json(), indent=2))

    for elem in get_response.json():
        current_id = elem.get('id')
        del_response = requests.delete(BASE_URL + "/" + target + "/" + str(current_id))
        #print(del_response)
        assert(status_code_successful(del_response.status_code))

    
    get_response = requests.get(BASE_URL + "/" + target)
    assert(status_code_successful(get_response.status_code))
    assert(len(get_response.json()) == 0)   

    
for elem in ["process-instance", "process-definition", "deployment", "decision-definition"]:
    delete_all_data(elem)


In [9]:
# deploy ONE example process instance
# TODO: extent for multiple

multipart_form_data = { 
    'deployment-name': (None, 'store'),
    'data': ('food_testing_2.bpmn', open('food_testing_2.bpmn', 'r')),
}

response = requests.post(BASE_URL + "/deployment/create", files=multipart_form_data)
assert(status_code_successful(response.status_code))

#print(json.dumps(response.json(), indent=2))
for elem in response.json().get('deployedProcessDefinitions'):
    new_process_id = elem

print(new_process_id)

food_testing_2:7:c9bf459e-6721-11ec-9f01-0242ac120004


In [98]:
# start instance
for x in range(10):
    headers = {'Content-Type': 'application/json'}
    response = requests.post("http://localhost:8080/engine-rest/process-definition/" + str(new_process_id)+ "/start", headers=headers)
    #print(json.dumps(response.json(), indent=2))
    assert(status_code_successful(response.status_code))


In [10]:
# get task and finish it
params = {
    'processDefinitionId': str(new_process_id)
    #'active': "true"
}
headers = {'Content-Type': 'application/json'}
response = requests.get(BASE_URL + "/task")
#print(json.dumps(response.json(), indent=2))
assert(status_code_successful(response.status_code))

for elem in response.json():
    task_id = elem.get('id')
    response = requests.post(BASE_URL + "/task/" + str(task_id) + "/complete", headers=headers)
    print(response)
    assert(status_code_successful(response.status_code))


<Response [204]>


In [95]:
# poll history service
# calculate durations of process instances by process definition

proc_def_response = requests.get(BASE_URL + "/history/process-definition/cleanable-process-instance-report")
assert(status_code_successful(proc_def_response.status_code))
#print(json.dumps(proc_def_response.json(), indent=2))

historic_df = pandas.DataFrame(columns=['processDefinitionId', 'processDefinitionKey', 'processDefinitionName', 'Duration'])


for elem in proc_def_response.json():
    processDefinitionId = elem.get('processDefinitionId')
    processDefinitionKey = elem.get('processDefinitionKey')
    processDefinitionName = elem.get('processDefinitionName')
    params = {'processDefinitionId': str(processDefinitionId)}
    proc_inst_response = requests.get(BASE_URL + "/history/process-instance", params=params)
    assert(status_code_successful(proc_inst_response.status_code))
    for elem2 in proc_inst_response.json():
        end_datetime = parser.parse(elem2.get('endTime'))
        start_datetime = parser.parse(elem2.get('startTime'))
        duration = (end_datetime - start_datetime)
        historic_df.loc[len(historic_df)] = [processDefinitionId, processDefinitionKey, processDefinitionName, duration]

print(historic_df)
print("Mean: " + str((historic_df.Duration.mean())))

                                 processDefinitionId processDefinitionKey  \
0  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
1  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
2  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
3  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
4  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
5  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
6  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
7  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
8  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   
9  food_testing_2:1:d60fb8ff-4a56-11ec-8c57-7e573...       food_testing_2   

  processDefinitionName               Duration  
0             Make food 0 days 00:03:17.496000  
1             Make food 0 days 00:03:17.436000  
2    

In [ ]:
# TODO
# next steps:
# create automatically running bp
# start it from here
# poll history service about it (to calculate rewards)